# 개인프로젝트 
###  #01.프로젝트 개요

## 관심사
- 관심있는 브랜드의 남성 스포츠 용품에 대한 정보 수집 : 미즈노, 아디다스 


        

        아디다스 남성 스포츠 용품 웹페이지 url : https://www.adidas.co.kr/men-performance?start=0


- 아디다스 크롤링 하나 데이터프레임(완료)+ 아디다스 크롤링 데이터 프레임 하나로  합치기 


In [40]:
# 필요한 모듈 참조
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame    

import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time


#### #02.웹 페이지 코드 수집 - 아디다스 


아디다스는 각 페이지 별로 상품 수집해서 그 전에는 이미지링크 + 상품 상세보기 링크 + 상품이름 -> 링크를 타고 들어가자 
그다음에 정가, 할인가 + 리뷰 수 

In [41]:

chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.implicitly_wait(5)

In [42]:

## 페이지마다 url 값이 48씩 늘어남
adidas_num = []
for i in range(0,5) :
    adidas_num.append(i*48)
print(adidas_num)



[0, 48, 96, 144, 192]


#### #03. 응답결과로부터 내용 추출하기

In [44]:

## 아디다스 빈 list
adidas_result_list =[]
# ## 페이지 수 만큼 반복  25페이지 까지 있음
# ## 수집할 컨텐츠가 있는 웹페이지의 주소 
# url = "https://www.adidas.co.kr/men-performance?start=0" 
for number in adidas_num :
    
    url = "https://www.adidas.co.kr/men-performance?start=%d" % number

    driver.get(url)
    time.sleep(1)
    ## 전체 HTML
    soup = BeautifulSoup(driver.page_source)
    adidas_list = soup.select(".grid-item")
    print("상품의 수 : ", len(adidas_list))
    print("%d 페이지" % ((number/48)+1))
    print("-" * 50)
    result_adidas_dict = {}
    
    for adidas in adidas_list:
        # 이미지링크 + 상품 상세보기 링크 + 상품이름
        
        # 상품이름
        adidas_nameEl = adidas.select(".glass-product-card__details > .glass-product-card__title")
        # print(adidas_nameEl)
        adidas_name = adidas_nameEl[0].text.strip()
        # print(adidas_name)

        ## 이미지 링크
        adidas_imageEl = adidas.select(".glass-product-card__assets > a > img")
        # print(adidas_imageEl)
        adidas_image = adidas_imageEl[0].attrs['src']
        # print(adidas_image)
            
        ## 상세보기 링크 

        adidas_detailEl = adidas.select(".glass-product-card > a ")
        adidas_detail = adidas_detailEl[0].attrs['href']
        
        ## url 접속할때 full 주소가 필요함. 
        if adidas_detail.find("https://www.adidas.co.kr") == -1 :
            adidas_detail = "https://www.adidas.co.kr" + adidas_detail

        # print(adidas_detail)

        ## 상세보기 페이지 들어가기 전에 가격이랑 할인가 할인금액=none 놓고 + 솔드아웃 표시
        ## if 써서 리뷰 태그 있으면 리뷰태그 , 없으면 none입니다 ㄱㄱ 
        adidas_priceEl = adidas.select(".gl-price .gl-price-item ")
        # print(len(adidas_priceEl)) ## -> 비어있으면 soldout 임 
        if len(adidas_priceEl) == 2 :
            adidas_BasePrice = adidas_priceEl[0].text.strip()
            adidas_LastPrice = adidas_priceEl[1].text.strip()
        elif len(adidas_priceEl) == 1:
            adidas_BasePrice = adidas_priceEl[0].text.strip()
            adidas_LastPrice = adidas_BasePrice  
        else :
            adidas_BasePrice = "재고없음"
            adidas_LastPrice = "재고없음"
            

        # print(adidas_BasePrice,adidas_LastPrice)
        ## 상세페이지 들어가서 리뷰개수 가져오기 ㄱㄱ 
        # ---------------------------------------------------
                    
        #### 상세보기 수집 (시작) 

        # ---------------------------------------------------
                    
        driver.get(adidas_detail)
        time.sleep(1)
        ## 상세보기 HTML            
        detailSoup = BeautifulSoup(driver.page_source)
        # print(detailSoup)
        # 정가, 할인금액,최종금액 + 리뷰 수 추출
        reviewEl = detailSoup.select(".reviews-header___1kFSZ h2")
        # print(reviewEl)   
        if len(reviewEl) ==1 :
            review = reviewEl[0].text.strip()
        else : 
            review = "(0)"
        # print(review)        
                    
        # ---------------------------------------------------
                    
        #### 상세보기 수집 (끝) 

        # ---------------------------------------------------
            
        result_adidas_dict = {"상품이름":adidas_name,"정가":adidas_BasePrice,
                                "최종가격":adidas_LastPrice,"리뷰수": review,"이미지링크": adidas_image}
        # print(result_adidas_dict)
        # print("-"*60)
        adidas_result_list.append(result_adidas_dict)

df_adidas = DataFrame(adidas_result_list)



상품의 수 :  48
1 페이지
--------------------------------------------------
상품의 수 :  48
2 페이지
--------------------------------------------------
상품의 수 :  48
3 페이지
--------------------------------------------------
상품의 수 :  48
4 페이지
--------------------------------------------------
상품의 수 :  48
5 페이지
--------------------------------------------------


In [49]:
## 아디다스 상품정보 데이터
df_adidas

,상품이름,정가,최종가격,리뷰수,이미지링크
0,울트라부스트 4.0 DNA,"199,000 원","199,000 원",리뷰 (29),"https://assets.adidas.com/images/w_383,h_383,f..."
1,M PL 티,"29,000 원","29,000 원",리뷰 (18),"https://assets.adidas.com/images/w_383,h_383,f..."
2,테크핏 에어로레디 트레이닝 롱 타이츠,"55,000 원","55,000 원",리뷰 (3),"https://assets.adidas.com/images/w_383,h_383,f..."
3,4크래프트 3 바 쇼트,"49,000 원","39,200 원",리뷰 (9),"https://assets.adidas.com/images/w_383,h_383,f..."
4,맨체스터 유나이티드 22/23 어웨이 저지,재고없음,재고없음,(0),"https://assets.adidas.com/images/w_383,h_383,f..."
...,...,...,...,...,...
235,오운더런 쉘 팬츠,재고없음,재고없음,(0),"https://assets.adidas.com/images/w_383,h_383,f..."
236,M 삼선 쇼츠,재고없음,재고없음,리뷰 (7),"https://assets.adidas.com/images/w_383,h_383,f..."
237,3S 에어로레디 러닝 트레이닝 베이스볼 캡,재고없음,재고없음,(0),"https://assets.adidas.com/images/w_383,h_383,f..."
238,티로 23 리그 트레이닝 팬츠,재고없음,재고없음,리뷰 (3),"https://assets.adidas.com/images/w_383,h_383,f..."


In [48]:
## xlsx 파일로 저장 

df_adidas.to_excel("아디다스_상품정보.xlsx",index=False)
